## Установка pandas

In [1]:
### !pip3 install pandas

In [1]:
import pandas as pd

## Чтение данных и фильтрация таблиц

In [18]:
user = pd.read_csv('user.csv')

In [19]:
user.birthday = pd.to_datetime(user.birthday, format='%d.%m.%Y')
user.last_visited = pd.to_datetime(user.last_visited, format='%d.%m.%Y %H:%M:%S')

In [20]:
user.head()

,id,gender,birthday,grade,region,last_visited
0,1,М,2007-11-02,9.0,Город Санкт-Петербург,2023-08-23 03:04:14
1,2,Ж,2009-01-29,8.0,Город Москва,2023-09-07 09:21:35
2,3,М,2007-05-09,NaN,Город Санкт-Петербург,2023-08-23 18:32:18
3,4,Ж,2007-09-27,9.0,Город Москва,2023-08-18 07:34:11
4,5,Ж,2007-08-04,NaN,Город Москва,2023-09-02 18:31:30


In [21]:
user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id            1000 non-null   int64         
 1   gender        1000 non-null   object        
 2   birthday      1000 non-null   datetime64[ns]
 3   grade         831 non-null    float64       
 4   region        1000 non-null   object        
 5   last_visited  1000 non-null   datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(1), object(2)
memory usage: 47.0+ KB


### Задание 1

Сколько в таблице `user` представлено учеников, родившихся в сентябре?

## НЕВЕРНО!!!   
Правильный ответ - 73. Получил в экселе

In [22]:
(user.birthday.dt.month == 9).sum()

73

In [13]:
user[user.birthday.dt.month == 9].head(50)

,id,gender,birthday,grade,region,last_visited
2,3,М,2007-09-05,NaN,Город Санкт-Петербург,2023-08-23 18:32:18
3,4,Ж,2007-09-27,9.0,Город Москва,2023-08-18 07:34:11
9,10,Ж,2009-09-28,7.0,Город Москва,2023-10-09 04:36:52
19,20,М,2007-09-13,9.0,Город Москва,2023-12-09 17:12:48
50,51,Ж,2007-09-19,NaN,Город Москва,2023-06-09 22:46:40
60,61,М,2008-09-04,9.0,Город Санкт-Петербург,2023-08-25 13:42:40
67,68,Ж,2010-09-05,7.0,Город Москва,2023-02-09 20:15:22
91,92,Ж,2006-09-27,10.0,Нижегородская область,2023-08-24 16:21:40
98,99,Ж,2010-09-08,7.0,Город Санкт-Петербург,2023-10-09 05:12:38
117,118,Ж,2007-09-14,NaN,Город Санкт-Петербург,2023-09-09 17:27:50


### Задание 2

Какое количество учеников в последний раз посещали платформу в первые $14$ дней сентября?

## НЕВЕРНО!!!

In [24]:
((user.last_visited.dt.month == 9) & (user.last_visited.dt.day.between(1, 14))).sum()

377

### Задание 3

На сколько в данных больше девятиклассников, чем девятиклассниц?

In [41]:
((user.gender == 'М') & (user.grade == 9)).sum() - ((user.gender == 'Ж') & (user.grade == 9)).sum()

5

### Задание 4

Какой процент учеников, у которых указан класс, обучается в $9$-м классе?

Ответ округлите до целых.

In [45]:
((user.grade == 9).sum() / user.grade.count() * 100).round()

24.0

### Задание 5

Сколько в данных представлено учеников из регионов, в названиях которых в качестве подстроки содержится слово «город»?

При расчёте статистики приведите названия регионов к строчному написанию.

In [72]:
user.region.str.lower().str.contains('город').sum()
# user[(user.region.str.contains('город')) & (user.gender == 'М')].shape

765

## Группировка, агрегация и соединение таблиц

In [ ]:
course = pd.read_csv('course.csv')

In [ ]:
user_course_progress = pd.read_csv('user_course_progress.csv')

**Относительным прогрессом** будем называть отношение суммарного числа баллов, которые ученик набрал за курс (колонка `progress` в таблице `user_course_progress`), к максимальному количеству баллов, которые ученик в принципе мог набрать за данный курс (колонка `max_progress` в таблице `course`).

С помощью представленных данных найдите ответ на следующий вопрос: в курсах какого предмета больше всего учеников, чей относительный прогресс не меньше $0.8$?

В рамках следующих заданий вам будет предложено, действуя поэтапно, найти ответ на этот вопрос.

### Задание 1

Произведите соединение таблиц `course` и `user_course_progress`.

В качестве ответа укажите количество строчек итоговой таблицы, в которых представлены ученики курсов по химии.

### Задание 2

Обратите внимание, что у части курсов нет значения в колонке `max_score`. Уберите из полученной в предыдщуем задании таблицы все строчки, в которых пропущено значение колонки `max_score`.

В качестве ответа укажите количество строчек в итоговой таблице.

### Задание 3

Добавьте в полученную в предыдущем задании таблицу колонку `relative_progress`, в которой для каждого ученика будет храниться значение его относительного прогресса в конкретном курсе.

В качестве ответа укажите среднее значение относительного прогресса в таблице. Ответ округлите до двух знаков после запятой с помощью функции `round`, которая встроена в Series.

### Задание 4

Оставьте в полученной в предыдущем задании таблице только те строчки, в которых значение относительного прогресса не меньше $0.8$.

В качестве ответа укажите число строчек в получившейся таблице.

### Задание 5

Произведите группировку строчек таблицы по предметам и посчитайте, сколько учеников с относительным прогрессом не меньше $0.8$ оказалось в каждом из предметов.

В качестве ответа заполните таблицу, в которой каждому предмету сопоставлено число таких учеников.

Если один и тот же ученик учился в разных курсах одного и того же предмета и в каждом из этих курсов достиг относительного прогресса не менее $0.8$, при подсчёте учтите его несколько раз.

### Задание 6

В курсах какого предмета больше всего учеников, достигших относительного прогресса не менее $0.8$?